In [1]:
from sklearn.model_selection import train_test_split
from pathlib import Path
import matplotlib.image as img
import json
import numpy as np
from tqdm import tqdm
import h5py

from multiprocessing import Pool
import time
from concurrent.futures import ThreadPoolExecutor
import functools
import pandas as pd

In [2]:
# Helper Functions from EDA
def ejson(p, fn): # extract json
    with open((p/fn).as_posix()) as f: return json.load(f) 
## Simple Json Reading Functions
def info_data(p): return ejson(p, 'info.json')
def dot_data(p): return ejson(p, 'dotInfo.json')
def frame_data(p): return ejson(p, 'frames.json')    
def screen_data(p): return ejson(p, 'screen.json')

def get_eye_info(i): return ejson(i, 'appleLeftEye.json'), ejson(i, 'appleRightEye.json')
def get_face_info(i): return ejson(i, 'appleFace.json')
def get_facegrid(i): return ejson(i, 'faceGrid.json')

def get_frame(p, img_fn): return img.imread(p/'..'/'..'/'gazecapture-224x224'/p.name/'frames'/img_fn)
## Larger Helper Functions
def coordinate_data(p): 
    data = dot_data(p)
    return data['XCam'], data['YCam'] # we want relative to camera coords
def orientation_data(p):
    sdata = screen_data(p)
    return sdata['Orientation']

In [3]:
# dset_path = Path('/media/brennan/Data/ml/gazecapture/')
dset_path = Path('../gazecapture/')

# Extract File Names For Keras Generator

In [4]:
test_size = 0.1

In [5]:
def extract_data(cases, accepted_o=[1, 2, 3, 4]):
    fnames = []
    XCam, YCam = [], []
    FaceH, FaceW, FaceX, FaceY = [], [], [], []
    IsValid = []
    
    i = 0
    for case in cases:
        FRAME_N = frame_data(case)
        O = orientation_data(case)
        XCAM, YCAM = coordinate_data(case)
#         LE, RE = get_eye_info(case)
        FACE = get_face_info(case)

        for frame_n, o, xcam, ycam, \
            fh, fw, fx, fy, valid in zip(FRAME_N, O, XCAM, YCAM, 
                                 FACE['H'], FACE['W'], FACE['X'], FACE['Y'], FACE['IsValid']):
            if o in accepted_o:
                fnames.append('{}/frames/{}'.format(case.name, frame_n))
                XCam.append(xcam)
                YCam.append(ycam)
                IsValid.append(valid)
                FaceH.append(fh)
                FaceW.append(fw)
                FaceX.append(fx)
                FaceY.append(fy)

    # package to dataframe
    df = pd.DataFrame(data={'file_names': fnames, 
                            'XCam': XCam,
                            'YCam': YCam,
                            'IsValid': IsValid,
                            'FaceH': FaceH, 
                            'FaceW': FaceW,
                            'FaceX': FaceX,
                            'FaceY': FaceY})        
    
    return df

(1326, 148)

In [ ]:
# Extract all case names first to split => faces it hasnt seen before
train_cases, test_cases = train_test_split(list(dset_path.iterdir()), test_size=test_size)
len(train_cases), len(test_cases)

# Extract all Images

In [23]:
# create dataframe from data
# ALL PHOTOS split into train / test sets
train_df = extract_data(train_cases)
test_df = extract_data(test_cases)

train_df.to_csv('train-df.csv')
test_df.to_csv('test-df.csv')

# Extract ONLY Portrait Images

In [12]:
train_df = extract_data(train_cases, accepted_o=[1])
test_df = extract_data(test_cases, accepted_o=[1])
# train_df.to_csv('portrait-train-df.csv')
# test_df.to_csv('portrait-test-df.csv')

In [13]:
len(test_df) / (len(test_df) + len(train_df)), len(train_df) / (len(test_df) + len(train_df))

(0.09879818605173366, 0.9012018139482664)

In [14]:
len(test_df), len(train_df)

(80958, 738470)

# Extract & Split LandScape Images

In [6]:
train_df = extract_data(train_cases, accepted_o=[3, 4])
test_df = extract_data(test_cases, accepted_o=[3, 4])
train_df.to_csv('landscape-train-df.csv')
test_df.to_csv('landscape-test-df.csv')

In [7]:
strain, sval = train_test_split(train_df, test_size=0.1)
strain.to_csv('landscape-traindf.csv'), sval.to_csv('landscape-valdf.csv')

(None, None)

## Stage 1 & 2 Split

In [ ]:
train_df_sample = train_df.sample(frac=0.5)
strain, sval = train_test_split(train_df_sample, test_size=0.1)
strain.to_csv('traindf_sample.csv'), sval.to_csv('valdf_sample.csv')

## Update Prev Stage Dataset

In [23]:
old_df = pd.read_csv('portrait-traindf.csv')
file_names = old_df['file_names'].values
cases_to_update = []

for fn in file_names:
    case = fn[:fn.index('/')]
    if not (dset_path / case in cases_to_update):
        cases_to_update.append(dset_path / case)

new_train_df = extract_data(cases_to_update, accepted_o=[1])
strain, sval = train_test_split(new_train_df, test_size=0.1)
strain.to_csv('portrait-traindf.csv'), sval.to_csv('portrait-valdf.csv')

(None, None)

# Export to hdf5
Not Used

In [10]:
# # Extracting and storing X values
# def extract_photo(f, case): return get_frame(case, f)

# def extract(case):
#     F = frame_data(case) # list of file names correlated to camera images 
#     with Pool(6) as p:  # multi core usale
#         x = p.map(functools.partial(extract_photo, case=case), F)
#     return x

# def extract_to_hdf5(cname, bs, extract_fcn): # bs dsets per hp5y file
#     f = h5py.File('{}-0.hdf5'.format(cname), 'w') 
#     for i, case in enumerate(list(dset_path.iterdir())):
#         cn = case.name
#         edata = extract_fcn(case)
        
#         dset = f.create_dataset('dset'+cn, (len(edata), 224, 224, 3))
#         dset[...] = edata

#         if i % bs == 0 and i!= 0:
#             f.close()
#             f = h5py.File('{}-{}.hdf5'.format(cname, i//bs), 'w')
# extract_to_hdf5('X', 500, extract)